In [6]:
import pysam as ps

In [7]:
def get_max_phase_set_id(record, samples):
    return max(int(record.samples[sample]['PS']) for sample in samples)

def get_multivariant_phasesets(vcf_path, samples=None):
    vcf = ps.VariantFile(vcf_path)
    tails = set()
    for record in vcf:
        phase_set_id = get_max_phase_set_id(record, vcf.header.samples)
        if phase_set_id != record.pos:
            tails.add((record.chrom, phase_set_id))
    vcf.close()
    vcf.open(vcf_path)
    result = []
    prev_phase_set_id = 0
    phase_set = []
    for record in vcf:
        phase_set_id = get_max_phase_set_id(record, vcf.header.samples)
        if phase_set_id == prev_phase_set_id:
            phase_set.append(record)
        elif (record.chrom, phase_set_id) in tails:
            if len(phase_set) > 0:
                result.append(phase_set)
            phase_set = [record]
            prev_phase_set_id = phase_set_id
    if len(phase_set) > 0:
        result.append(phase_set)
    return result

def is_somatic(record):
    return 'SOMATIC' in record.info

def is_denovo(record):
    return 'SOMATIC' in record.info

def is_passed(record):
    return 'PASS' in record.filter

def all_passed(records):
    return all(is_passed(record) for record in records)

def is_heterozygous(record, sample):
    genotype = record.samples[sample]['GT']
    return sum(1 if allele is None or allele > 0 else 0 for allele in genotype) < len(genotype)

def any_heterozygous(record, samples):
    return any(is_heterozygous(record, sample) for sample in samples)

In [8]:
vcf_path = "/Users/dcooke/Genomics/octopus/paper/somatic/paired/calls/skin/N30/T60/octopus.all.vcf.gz"

In [4]:
phase_sets = get_multivariant_phasesets(vcf_path)

In [5]:
len(phase_sets)

1062208

In [6]:
somatic_phasesets = []
for phaseset in phase_sets:
    if any(is_somatic(record) for record in phaseset):
        somatic_phasesets.append(phaseset)

In [7]:
len(somatic_phasesets)

133039

In [8]:
passed_somatic_phasesets = []
for phaseset in phase_sets:
    if any(is_somatic(record) for record in phaseset) and all_passed(phaseset):
        passed_somatic_phasesets.append(phaseset)

In [9]:
len(passed_somatic_phasesets)

90121

In [10]:
[print(rec) for rec in passed_somatic_phasesets[113]]

1	4737319	.	G	A	53.54	PASS	AC=1;AN=5;DP=80;MP=19.16;MQ=60;MQ0=0;NS=2;PP=53.54;SOMATIC	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	0|0:401:25:60:4737319:99:.:.,.:24.77:PASS	0|0|1:401:55:60:4737319:99:0.14:0.085,0.2:24.77:PASS

1	4737405	.	T	C	1324.38	PASS	AC=2;AN=5;DP=82;MP=19.16;MQ=60;MQ0=0;NS=2;PP=1324.38	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|0:300:24:60:4737319:99:23.8:PASS	1|0|0:300:58:60:4737319:99:17.73:PASS



[None, None]

In [9]:
def count_somatics(records):
    return sum(is_somatic(record) for record in records)

In [12]:
sum(count_somatics(phase_set) > 1 for phase_set in passed_somatic_phasesets)

4872

In [30]:
sum(count_somatics(phase_set) if count_somatics(phase_set) > 1 else 0 for phase_set in passed_somatic_phasesets)

9834

In [16]:
sum(count_somatics(phase_set) > 2 for phase_set in passed_somatic_phasesets)

87

In [17]:
sum(count_somatics(phase_set) > 3 for phase_set in passed_somatic_phasesets)

3

In [20]:
for phase_set in passed_somatic_phasesets:
    if count_somatics(phase_set) > 3:
        for rec in phase_set:
            print(rec)

8	139011863	.	G	GA	81.19	PASS	AC=2;AN=6;DP=116;MP=3233.06;MQ=59;MQ0=0;NS=2;PP=81.19;SOMATIC	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	0|0:264:43:59:139011863:99:.:.,.:14.27:PASS	0|0|1|1:264:73:60:139011863:99:0.13:0.088,0.19:3.43:PASS

8	139011865	.	AT	A	81.19	PASS	AC=2;AN=2;DP=121;MP=3233.06;MQ=59;MQ0=0;NS=2;PP=81.19;SOMATIC	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	.|.:160:43:59:139011863:99:.:.,.:3.59:PASS	.|.|1|1:160:78:60:139011863:99:0.13:0.088,0.19:3.13:PASS

8	139011866	.	T	A,*	1488.57	PASS	AC=4,2;AN=6;DP=121;MP=3233.06;MQ=59;MQ0=0;NS=2;PP=1488.57	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|1:127:43:59:139011863:99:56.16:PASS	1|1|2|2:127:78:60:139011863:99:5.94:PASS

8	139011931	.	C	T	81.19	PASS	AC=2;AN=6;DP=137;MP=3233.06;MQ=59;MQ0=0;NS=2;PP=81.19;SOMATIC	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	0|0:293:53:59:139011863:99:.:.,.:6.62:PASS	0|0|1|1:293:84:60:139011863:99:0.13:0.088,0.19:3.78:PASS

8	139011951	.	G	GA	81.19	PASS	AC=1;AN=6;DP=123;MP=3233.06;MQ=59;MQ0=0;NS=2;PP=81.19;SOMATIC	G

In [13]:
sum(count_somatics(phase_set) == 1 for phase_set in passed_somatic_phasesets)

85249

In [14]:
passed_somatic_phasesets_with_het_germline = []
for phase_set in passed_somatic_phasesets:
    if any(not is_somatic(record) and is_heterozygous(record, "NA12878.NORMAL") for record in phase_set):
        passed_somatic_phasesets_with_het_germline.append(phase_set)

In [33]:
passed_somatics_phasesets_with_het_germline = [phase_set for phase_set in passed_somatic_phasesets_with_het_germline if count_somatics(phase_set) > 1]

In [36]:
[print(rec) for rec in passed_somatics_phasesets_with_het_germline[11]]

1	56766830	.	T	A	1760.94	PASS	AC=3;AN=6;DP=95;MP=82.82;MQ=60;MQ0=0;NS=2;PP=794.81	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|0:795:39:60:56766830:99:31.54:PASS	1|0|1|0:795:56:60:56766830:99:13.33:PASS

1	56766833	.	G	A	1121.17	PASS	AC=1;AN=6;DP=96;MP=82.82;MQ=60;MQ0=0;NS=2;PP=1121.17;SOMATIC	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	0|0:454:37:60:56766830:99:.:.,.:16.32:PASS	0|0|1|0:454:59:60:56766830:99:0.44:0.36,0.53:19.44:PASS

1	56766879	.	G	A	717.51	PASS	AC=1;AN=6;DP=97;MP=82.82;MQ=60;MQ0=0;NS=2;PP=717.51;SOMATIC	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	0|0:718:32:60:56766830:99:.:.,.:23.98:PASS	0|0|0|1:718:65:60:56766830:99:0.44:0.13,0.28:29.54:PASS



[None, None, None]

In [21]:
sum(count_somatics(phase_set) for phase_set in passed_somatic_phasesets_with_het_germline)

57217

In [24]:
len(passed_somatic_phasesets_with_het_germline)

55776

In [40]:
def phase_set_len(phase_set):
    return phase_set[-1].pos - phase_set[0].pos

In [41]:
passed_somatic_phasesets_with_het_germline_lengths = [phase_set_len(phase_set) for phase_set in passed_somatic_phasesets_with_het_germline]

NameError: name 'passed_somatic_phasesets_with_het_germline' is not defined

In [23]:
sum(count_somatics(phase_set) > 1 for phase_set in passed_somatic_phasesets_with_het_germline)

1407

In [31]:
sum(count_somatics(phase_set) if count_somatics(phase_set) > 1 else 0 for phase_set in passed_somatic_phasesets_with_het_germline)

2848

In [27]:
[print(rec) for rec in passed_somatic_phasesets_with_het_germline[10]]

1	2282135	.	G	A	1178	PASS	AC=2;AN=5;DP=72;MP=27.68;MQ=59;MQ0=0;NS=2;PP=1178	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|0:319:30:60:2282135:49:24.77:PASS	1|0|0:319:42:59:2282135:49:13.54:PASS

1	2282204	.	G	A	865.89	PASS	AC=3;AN=5;DP=71;MP=27.68;MQ=60;MQ0=0;NS=2;PP=412.25	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	0|1:319:25:60:2282135:49:20.81:PASS	0|1|1:319:46:60:2282135:49:12.31:PASS

1	2282263	.	G	A	392.22	PASS	AC=1;AN=5;DP=74;MP=27.68;MQ=60;MQ0=0;NS=2;PP=392.22;SOMATIC	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	0|0:658:29:60:2282135:49:.:.,.:24.77:PASS	0|0|1:658:45:60:2282135:49:0.33:0.28,0.38:22.73:PASS

1	2282373	.	C	T	1088.61	PASS	AC=3;AN=5;DP=74;MP=27.68;MQ=59;MQ0=0;NS=2;PP=688.86	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	0|1:319:27:60:2282135:49:57.49:PASS	0|1|1:319:47:59:2282135:49:8.68:PASS



[None, None, None, None]

In [28]:
57217 / 257930

0.22183150467181018

In [29]:
4872 / 257930

0.01888884581087892

In [10]:
import subprocess as sp
from os import remove

vcfanno_bin = '/Users/dcooke/Genomics/apps/vcfanno'

def write_vcfanno_haplotype_config(vaf_vcf, out):
    with open(out, 'w') as toml:
        toml.write('[[annotation]]\n')
        toml.write('file="' + vaf_vcf + '"\n')
        toml.write('fields=["HAP"]\n')
        toml.write('ops=["self"]\n')
        toml.write('names=["HAP"]\n')

def run_vcfanno(in_vcf, vcfanno_config, out_vcf):
    vcfanno_cmd = [vcfanno_bin, '-permissive-overlap', vcfanno_config, in_vcf]
    bgzip_cmd = ['bgzip']
    vcfanno = sp.Popen(vcfanno_cmd, stdout=sp.PIPE)
    out = open(out_vcf, 'w')
    bgzip = sp.Popen(bgzip_cmd, stdin=vcfanno.stdout, stdout=out)
    vcfanno.stdout.close()
    output = bgzip.communicate()[0]

def annotate_haplotypes(caller_vcf, truth_vcf, out_vcf):
    vcfanno_config = 'hap.toml'
    write_vcfanno_haplotype_config(truth_vcf, vcfanno_config)
    run_vcfanno(caller_vcf, vcfanno_config, out_vcf)
    remove(vcfanno_config)

In [11]:
truth_vcf = '/Users/dcooke/Genomics/octopus/paper/somatic/truth/skin.vcf.gz'
hap_annotated_vcf = vcf_path.replace('.vcf', '.HAP.tmp.vcf')
annotate_haplotypes(vcf_path, truth_vcf, hap_annotated_vcf)
hap_annotated_phase_sets = get_multivariant_phasesets(hap_annotated_vcf)

In [12]:
passed_somatic_hap_annotated_phase_sets = [phaseset for phaseset in hap_annotated_phase_sets if any(is_somatic(record) for record in phaseset) and all_passed(phaseset)]

In [13]:
passed_somatic_hap_annotated_phase_sets_with_germline_het = [phase_set for phase_set in passed_somatic_hap_annotated_phase_sets if any(not is_somatic(record) and is_heterozygous(record, "NA12878.NORMAL") for record in phase_set)]

In [14]:
len(passed_somatic_hap_annotated_phase_sets_with_germline_het)

55776

In [15]:
for rec in passed_somatic_hap_annotated_phase_sets_with_germline_het[0]:
    print(rec)

1	55063	.	C	A	448.5	PASS	AC=1;AN=5;DP=48;MP=8.84;MQ=39;MQ0=0;NS=2;PP=435.4;SOMATIC;HAP=1	GT:GQ:DP:MQ:PS:PQ:MAP_VAF:VAF_CR:RFQUAL:FT	0|0:435:25:39:55063:99:.:.,.:9.52:PASS	0|0|1:435:23:39:55063:99:0.62:0.53,0.71:7.19:PASS

1	55085	.	T	A	323.6	PASS	AC=2;AN=5;DP=43;MP=8.84;MQ=39;MQ0=0;NS=2;PP=323.63	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|0:154:21:39:55063:99:19.88:PASS	1|0|0:154:22:39:55063:99:18.36:PASS

1	55164	.	C	A	484.8	PASS	AC=5;AN=5;DP=33;MP=8.84;MQ=36;MQ0=0;NS=2;PP=289.19	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|1:154:12:38:55063:99:22.29:PASS	1|1|1:154:21:36:55063:99:27.6:PASS

1	55299	.	C	T	1452.5	PASS	AC=5;AN=5;DP=48;MP=8.84;MQ=48;MQ0=0;NS=2;PP=943.46	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|1:154:20:48:55063:99:36.66:PASS	1|1|1:154:28:48:55063:99:36.66:PASS

1	55326	.	T	C	1720.3	PASS	AC=5;AN=5;DP=54;MP=8.84;MQ=49;MQ0=0;NS=2;PP=1035.54	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|1:154:19:49:55063:99:28.67:PASS	1|1|1:154:35:50:55063:99:36.66:PASS



In [16]:
consensus_vcf1_name = '/Users/dcooke/Genomics/syntumour/consensus/octopus.recall.NA12878.60x.bwa.b37.rf-hard-filtered.consensus1.vcf.gz'
consensus_vcf2_name = '/Users/dcooke/Genomics/syntumour/consensus/octopus.recall.NA12878.60x.bwa.b37.rf-hard-filtered.consensus2.vcf.gz'

In [17]:
hap1_germline_hets, hap2_germline_hets = [], []
for phase_set in passed_somatic_hap_annotated_phase_sets_with_germline_het:
    if count_somatics(phase_set) == 1:
        hap = 0
        for rec in phase_set:
            if is_somatic(rec):
                if 'HAP' in rec.info:
                    hap = rec.info['HAP']
                else:
                    continue
        for rec in phase_set:
            if not is_somatic(rec) and is_heterozygous(rec, 'NA12878.NORMAL'):
                tumour_gt = rec.samples['NA12878.TUMOUR']['GT']
                if len(tumour_gt) == 3:
                    if tumour_gt[-1] == 0:
                        if hap == 0:
                            hap2_germline_hets.append(rec)
                        else:
                            hap1_germline_hets.append(rec)
                    elif tumour_gt[-1] == 1:
                        if hap == 0:
                            hap1_germline_hets.append(rec)
                        else:
                            hap2_germline_hets.append(rec)

In [18]:
print(hap2_germline_hets[0])

1	1072636	.	T	G	168.7	PASS	AC=2;AN=5;DP=68;MP=19.81;MQ=59;MQ0=0;NS=2;PP=168.73	GT:GQ:DP:MQ:PS:PQ:RFQUAL:FT	1|0:81:27:59:1072244:99:13.74:PASS	1|0|0:81:41:60:1072244:99:5.88:PASS



In [19]:
len(hap1_germline_hets), len(hap2_germline_hets)

(42456, 42408)

In [34]:
from os import remove
from os.path import exists

def index_vcf(vcf_path):
    sp.call(['tabix', vcf_path])
    return vcf_path + '.tbi'

def remove_vcf(vcf_path):
    remove(vcf_path)
    if exists(vcf_path + '.tbi'):
        remove(vcf_path + '.tbi')

In [36]:
phase_errors = []

tmp_header = ps.VariantFile(vcf_path).header

hap1_germline_hets_tmp_vcf_name = 'hap1_germline_tmp.vcf.gz'
hap1_germline_hets_tmp_vcf = ps.VariantFile(hap1_germline_hets_tmp_vcf_name, 'w', header=tmp_header)
for rec in hap1_germline_hets:
    hap1_germline_hets_tmp_vcf.write(rec)
hap1_germline_hets_tmp_vcf.close()
index_vcf(hap1_germline_hets_tmp_vcf_name)
hap2_germline_hets_tmp_vcf_name = 'hap2_germline_tmp.vcf.gz'
hap2_germline_hets_tmp_vcf = ps.VariantFile(hap2_germline_hets_tmp_vcf_name, 'w', header=tmp_header)
for rec in hap2_germline_hets:
    hap2_germline_hets_tmp_vcf.write(rec)
hap2_germline_hets_tmp_vcf.close()
index_vcf(hap2_germline_hets_tmp_vcf_name)

fp_hap1_hets_vcf_name = 'fp_hap1_hets.vcf.gz'
sp.call(['bcftools', 'isec', '-C', '-n=1', '-w1', '-Oz', '-o', fp_hap1_hets_vcf_name, hap1_germline_hets_tmp_vcf_name, consensus_vcf1_name])
index_vcf(fp_hap1_hets_vcf_name)
hap1_phase_errors_vcf_name = 'hap1_phase_errors.vcf.gz'
sp.call(['bcftools', 'isec', '-n=2', '-w1', '-Oz', '-o', hap1_phase_errors_vcf_name, fp_hap1_hets_vcf_name, consensus_vcf2_name])
remove_vcf(fp_hap1_hets_vcf_name)

hap1_phase_errors_vcf = ps.VariantFile(hap1_phase_errors_vcf_name)
phase_errors += [rec for rec in hap1_phase_errors_vcf]
hap1_phase_errors_vcf.close()
remove_vcf(hap1_phase_errors_vcf_name)

fp_hap2_hets_vcf_name = 'fp_hap2_hets.vcf.gz'
sp.call(['bcftools', 'isec', '-C', '-n=1', '-w1', '-Oz', '-o', fp_hap2_hets_vcf_name, hap2_germline_hets_tmp_vcf_name, consensus_vcf2_name])
index_vcf(fp_hap2_hets_vcf_name)
hap2_phase_errors_vcf_name = 'hap1_phase_errors.vcf.gz'
sp.call(['bcftools', 'isec', '-n=2', '-w1', '-Oz', '-o', hap2_phase_errors_vcf_name, fp_hap2_hets_vcf_name, consensus_vcf1_name])
remove_vcf(fp_hap2_hets_vcf_name)

hap2_phase_errors_vcf = ps.VariantFile(hap2_phase_errors_vcf_name)
phase_errors += [rec for rec in hap2_phase_errors_vcf]
hap2_phase_errors_vcf.close()
remove_vcf(hap1_phase_errors_vcf_name)

remove_vcf(hap1_germline_hets_tmp_vcf_name)
remove_vcf(hap2_germline_hets_tmp_vcf_name)

In [39]:
100 * (1. - len(phase_errors) / (len(hap1_germline_hets) + len(hap2_germline_hets)))

92.62467006033182